# To Contracept or Not to Contracept

Import packeges and py-files

In [1]:
# a. set magic 
%load_ext autoreload
%autoreload 2

# b. packeges 
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import copy

# c. import py-files 
from Model import child_model 
import Solve as solver
import Estimate as estimate

## Set up

### Read data

In [2]:
model = child_model()
dta = model.read_data()

### State trantision conditional on age and choice from the real data

In [3]:
solver.P_list(model, dta)

## Show that the estimating works on simulated data

### Solve the model and simulate data

In [4]:
# a. Solve model 
V, pnc = solver.BackwardsInduction(model)

# b. Simulate data
data = model.sim_data(pnc)

### Chosen parameters

In [5]:
print(model.mu, model.eta1, model.eta2)

0.4 0.8 -0.3


### Estimate parameters from simulated data

In [6]:
# a. Initialize parameters 
samplesize = data.shape[0]
par_mu = copy.copy(model.mu)
par_eta1 = copy.copy(model.eta1)
par_eta2 = copy.copy(model.eta2)

# b. Number of gridpoints (must be the same size)
I = 50

# c. Search grid 
log_lik = np.nan + np.zeros((I,I,I))
mu = np.linspace(0,1.8,I)
eta1 = np.linspace(0,1.8,I)
eta2 = np.linspace(-0.9,0.9,I)

# d. Loop over all parameter grids 
for i in range(I):
    for j in range(I):
        for k in range(I):
                # i. Estimate parameters
                pnames = ['eta1', 'eta2', 'mu']
                theta = np.array([eta1[i], eta2[j], mu[k]])
                # ii. Estimate log-likelihood in simulated data
                log_lik[i,j,k] = estimate.ll(theta, model, solver, data, pnames)

# e. Log-likelihood 
log_lik *= samplesize*(-1)

# f. Re-inset the true parameters 
model.eta1 = copy.copy(par_eta1)
model.eta2 = copy.copy(par_eta2)
model.mu = copy.copy(par_mu)

# g. Find the maximum log-likelihood
x, y, z = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# h. Print results
print(mu[z], eta1[x], eta2[y])

0.40408163265306124 0.8816326530612246 -0.31224489795918364


## Estimate parameters on real-life data

In [7]:
# a. Initialize parameters 
samplesize = dta.shape[0]
par_mu = copy.copy(model.mu)
par_eta1 = copy.copy(model.eta1)
par_eta2 = copy.copy(model.eta2)

# b. Number of gridpoints
I = 50

# c. Search grid 
log_lik = np.nan + np.zeros((I,I,I))
mu = np.linspace(0,1.8,I)
eta1 = np.linspace(0,1.8,I)
eta2 = np.linspace(-0.9,0.9,I)

# d. Loop over all parameter grids 
for i in range(I):
    for j in range(I):
        for k in range(I):
                
                # i. Estimate parameters
                pnames = ['eta1', 'eta2', 'mu']
                theta = np.array([eta1[i], eta2[j], mu[k]])
                # ii. Estimate log-likelihood in real data
                log_lik[i,j,k] = estimate.ll(theta, model, solver, dta, pnames)

# e. Log-likelihood 
log_lik *= samplesize*(-1)

# f. Re-inset the true parameters 
model.eta1 = copy.copy(par_eta1)
model.eta2 = copy.copy(par_eta2)
model.mu = copy.copy(par_mu)

# g. Find the maximum log-likelihood
x, y, z = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# h. Print results
print(mu[z], eta1[x], eta2[y])

0.8081632653061225 0.07346938775510205 0.1653061224489797


### Update parameters

In [8]:
model.mu = mu[z]
model.eta1 = eta1[x]
model.eta2 = eta2[y]

### Solve the model and simulate data with true parameters

In [9]:
# a. Solve model
V, pnc = solver.BackwardsInduction(model)

# b. Simulate data
data = model.sim_data(pnc)

### Distribution of children

In [10]:
# a. Data in last period 
data0 = data[(data['t']==model.T-1)]

# b. Share of children 
data0.x.value_counts()/model.N*100

2    40.101892
3    28.857351
1    15.502183
4    14.665211
0     0.873362
Name: x, dtype: float64

### Choice probabilities

In [11]:
print(pnc)

[[0.338 0.376 0.371 0.383 0.395 0.438 0.429 0.458 0.486 0.487 0.518 0.542
  0.543 0.515 0.512 0.514 0.478 0.467 0.395 0.363 0.346 0.351 0.325 0.332
  0.316 0.315 1.   ]
 [0.147 0.113 0.155 0.155 0.162 0.149 0.177 0.18  0.192 0.218 0.232 0.257
  0.285 0.31  0.332 0.354 0.364 0.377 0.356 0.346 0.339 0.351 0.328 0.345
  0.323 0.325 1.   ]
 [0.059 0.028 0.053 0.049 0.049 0.034 0.052 0.048 0.051 0.07  0.075 0.09
  0.117 0.16  0.19  0.22  0.262 0.294 0.318 0.328 0.333 0.352 0.332 0.357
  0.33  0.335 1.   ]
 [0.064 0.027 0.047 0.041 0.037 0.022 0.034 0.029 0.029 0.04  0.04  0.047
  0.064 0.097 0.119 0.142 0.187 0.222 0.279 0.307 0.323 0.349 0.335 0.369
  0.337 0.345 1.   ]
 [0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308
  0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308
  0.308 0.308 1.   ]]


## Counter factuals

### Perfect contraceptive use

In [12]:
# a. Perfect contraceptive use 
model.p2_list = np.ones([model.T,2])*np.array([1, 0])

# b. Solve model
V_cf, pnc_cf = solver.BackwardsInduction(model)

# c. Simulate data
data_cf = model.sim_data(pnc_cf)

### Distrubution of children

In [15]:
# a. Data in last period
data_cf0 = data_cf[(data_cf['t']==model.T-1)]

# b. Share of children
data_cf0.x.value_counts()/model.N*100

2    47.088792
3    24.090247
1    21.579330
4     5.967977
0     1.273654
Name: x, dtype: float64


### Choice probabilities

In [14]:
print(pnc_cf)

[[0.44  0.474 0.438 0.439 0.447 0.491 0.468 0.496 0.523 0.518 0.544 0.572
  0.566 0.54  0.531 0.536 0.499 0.477 0.405 0.374 0.348 0.354 0.326 0.332
  0.316 0.315 1.   ]
 [0.151 0.139 0.172 0.177 0.179 0.168 0.194 0.197 0.208 0.231 0.247 0.269
  0.296 0.319 0.339 0.362 0.372 0.382 0.361 0.353 0.342 0.354 0.33  0.345
  0.323 0.325 1.   ]
 [0.024 0.018 0.037 0.04  0.039 0.029 0.047 0.045 0.048 0.066 0.074 0.086
  0.116 0.155 0.187 0.217 0.259 0.294 0.32  0.332 0.335 0.355 0.334 0.357
  0.33  0.335 1.   ]
 [0.009 0.005 0.015 0.016 0.016 0.009 0.019 0.017 0.017 0.027 0.03  0.035
  0.053 0.082 0.107 0.129 0.176 0.218 0.277 0.307 0.325 0.352 0.337 0.369
  0.337 0.345 1.   ]
 [0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308
  0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308 0.308
  0.308 0.308 1.   ]]
